In [49]:
# %pip install --force-reinstall --no-deps torch==2.2.2 torchvision==0.17.2

In [50]:
# %pip install statsforecast mlforecast pandas polars pyarrow holidays lightgbm neuralforecast

In [51]:
import pandas as pd
from utils.constants import FREQ_DAILY, FREQ_MONTHLY, FAMILY_BASELINE, FAMILY_STATISTICAL, FAMILY_MACHINE_LEARNING, FAMILY_DEEP_LEARNING, SPLIT_VAL
from utils.preprocessing import load_daily_data, load_monthly_data
from utils.baseline import run_baseline_forecast_daily, run_baseline_forecast_monthly
from utils.statistical import run_statistical_forecast_daily, run_statistical_forecast_monthly
from utils.machine_learning import run_machine_learning_forecast_daily, run_machine_learning_forecast_monthly
from utils.deep_learning import run_deep_learning_forecast_daily, run_deep_learning_forecast_monthly
from utils.metrics import calculate_metrics

### Loading Data:

In [52]:
daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

### Forecasts:

##### Baseline Forecast:

In [53]:
base_daily_val, base_daily_test = run_baseline_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
base_monthly_val, base_monthly_test = run_baseline_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Statistical Forecast:

In [54]:
statis_daily_val, statis_daily_test = run_statistical_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
statis_monthly_val, statis_monthly_test = run_statistical_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Machine Learning Forecast:

In [55]:
ml_daily_val, ml_daily_test = run_machine_learning_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
ml_monthly_val, ml_monthly_test = run_machine_learning_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Deep Learning Forecast:

In [56]:
dl_daily_val, dl_daily_test = run_deep_learning_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
dl_monthly_val, dl_monthly_test = run_deep_learning_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

### Metrics:

##### Baseline Metrics:

In [57]:
base_metrics = pd.concat([
    calculate_metrics(base_daily_val, FAMILY_BASELINE, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(base_monthly_val, FAMILY_BASELINE, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
]).reset_index(drop=True).sort_values(by=["MAPE"])

##### Statistical Metrics:

In [58]:
stat_metrics = pd.concat([
    calculate_metrics(statis_daily_val, FAMILY_STATISTICAL, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(statis_monthly_val, FAMILY_STATISTICAL, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
]).reset_index(drop=True).sort_values(by=["MAPE"])

##### Machine Learning Metrics:

In [59]:
ml_metrics = pd.concat([
    calculate_metrics(ml_daily_val, FAMILY_MACHINE_LEARNING, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(ml_monthly_val, FAMILY_MACHINE_LEARNING, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
]).reset_index(drop=True).sort_values(by=["MAPE"])

##### Deep Learning Metrics:

In [60]:
dl_metrics = pd.concat([
    calculate_metrics(dl_daily_val, FAMILY_DEEP_LEARNING, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(dl_monthly_val, FAMILY_DEEP_LEARNING, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
]).reset_index(drop=True).sort_values(by=["MAPE"])

##### Overall metrics:

In [61]:
overall_metrics = pd.concat([base_metrics, stat_metrics, ml_metrics, dl_metrics]).reset_index(drop=True).sort_values(by=["MAPE"])

In [62]:
overall_metrics

,Model,Family,Frequency,Split,MAE,RMSE,MAPE
16,LGBMRegressor_Lag,ml,D,val,16.875428,21.555135,0.156086
17,RandomForestRegressor_Lag,ml,MS,val,16.372535,19.384507,0.168571
18,LinearRegression,ml,MS,val,24.084483,36.635060,0.199615
0,SeasonalNaive,base,MS,val,24.026788,32.030024,0.218963
1,Structural,base,MS,val,20.621831,23.950648,0.223417
19,LinearRegression,ml,D,val,26.217516,35.935484,0.228100
8,AutoETS,stat,D,val,20.829845,23.764168,0.229846
2,HistoricAverage,base,D,val,26.302547,33.423877,0.230858
3,HistoricAverage,base,MS,val,26.334404,33.468767,0.231082
9,HoltWinters,stat,D,val,20.827490,24.143708,0.232652
